# Предобработка данных

Автор ноутбука - Дуркин Анатолий Альбертович

Преподаватель кафедры прикладной математики и компьютерных наук СГУ им. Питирима Сорокина

Замечания, предложения, идеи, вопросы, связь с автором:
- anatoliy.durkin@mail.ru
- Telegram - @AnatoDu

Больше информации и материалов на канале автора: https://t.me/smth_on_it

## Красота данных

Данные редко бывают чистыми и красивыми, обычно в них присутствуют ошибки, пропуски и аномальные значения. И всё это необходимо обрабатывать до того, как приступить к анализу или построению моделей машинного обучения. Помните: мусор на входе - мусор на выходе. Если мы возьмём такие необработанные данные для анализа, наши выводы будут как минимум не совсем верны, а как макисмум - абсолютно противоположны реальности.

Далее рассмотрим основные проблемы, которые могут встретиться в данных, а также методы обработки данных. Этот список (проблем и методов) совершенно не исчерпывающий, но дающий базовое понимание.

In [ ]:
import pandas as pd

## Наименования столбцов

Посмотрим на следующие данные.

In [ ]:
df_cols = pd.read_csv('survey.csv')

In [ ]:
df_cols.info()

In [ ]:
df_cols.columns

В чём проблема этих данных? Да, их может быть много, но сейчас мы обратим внимание на то, что первые столбцы написаны с большой буквы, а все остальные с маленькой. Разве это проблема?

В процессе анализа, когда вы будете постоянно обращаться к тем или иным столбцам, очень легко забыть, какой из них пишется с большой буквы, а какой с маленькой, какой с нижним подчеркиванием, а какой с пробелом или дефисом, да и вообще какое странное название было у какого-нибудь столбца.

Поэтому рекомендуется всегда приводить названия столбцов к единообразному виду и по возможности давать столбцам осмысленные названия. Удобно, когда они написаны маленькими буквами и с нижним подчеркиванием на месте пробелов. Конечно, их можно привести и к другому единообразию, всё зависит от ваших предпочтений и изначального вида заголовков. А возможно, что вам вообще не захочется этого делать - дело ваше, я не настаиваю. Главное не запутайтесь!

Если же решите переименовать - есть два способа. Первый позволяет задать полностью список заголовков. Удобно, когда надо поменять все или почти все названия.

In [ ]:
df_cols.columns = ['timestamp', 'age', 'gender', 'country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments']

In [ ]:
df_cols.columns

Действительно, теперь у нас новые заголовки.

Но в данном случае проще было переименовать только первые столбцы. Посмотрим, как это происходит. Для этого заново загрузим данные и воспользуемся другим методом переименования.

In [ ]:
df_cols = pd.read_csv('survey.csv')

In [ ]:
df_cols.columns

In [ ]:
df_cols.rename({'Timestamp': 'datetime', 'Age': 'age', 'Gender': 'gender', 'Country': 'country'},
           axis=1, inplace=True)

In [ ]:
df_cols.columns

И тут сработало! Это значительно удобнее, когда надо изменить буквально пару названий.

А теперь перейдём к другим данным и другим проблемам в данных.

## Знакомство с данными

Данные можно загружать не только из файлов, лежащих где-то рядом с jupyter-ноутбуком, но даже из сети.

Загрузим данные от Яндекс Практикума (не уверен в том, что их можно использовать, но уточню этот вопрос и заменю их на похожие, так что данные могут поменяться).

Это информация для оценки надежности заёмщика. Получим данные и познакомимся с ними.

In [ ]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

В датасете присутствуют следующие столбцы:

- children – количество детей в семье;
- days_employed – общий трудовой стаж в днях;
- dob_years – возраст клиента в годах;
- education – уровень образования клиента;
- education_id – идентификатор уровня образования;
- family_status – семейное положение;
- family_status_id – идентификатор семейного положения;
- gender – пол клиента;
- income_type – тип занятости;
- debt – имел ли задолженность по возврату кредитов (1 – имел, 0 – не имел);
- total_income – ежемесячный доход;
- purpose – цель получения кредита.

Первичное знакомство с данными необходимо, чтобы посмотреть, как они устроены и, возможно, сразу определить некоторые проблемы.

In [ ]:
df

Да, в этих данных уже можно заметить некоторые проблемы. В стаже почему-то записаны отрицательные значения, что явно противоречит логике, а также какие-то очень большие значения, вызывающие сомнения. Ещё можно заметить, что в столбце образования есть слова "среднее" и "Среднее". Для нас это, конечно, одно значения. но для компьютера это разные вещи. На это тоже стоит обратить внимание. Видите ли вы ещё какие-либо сомнительные вещи?

Посмотрим на основную информацию.

In [ ]:
df.info()

Здесь мы можем видеть все столбцы нашего датасета, количество непустых значений в каждом и его тип. Напоминаю, что `object` - это строковый тип данных в pandas.

Отсюда можно получить информацию, что в двух столбцах имеются пустые значения, их больше двух тысяч, и их одинаковое количество.

Также посмотрим на основные статистические метрики численных данных.

In [ ]:
df.describe()

В этой таблице можно увидеть несколько странностей. У кого-то есть -1 ребёнок, чего точно не может быть, а у кого-то их 20. Больше трёх четвертей всех данных по стажу отрицательны, а максимальное значение подозрительно большое. А в возрасте есть люди с нулевым возрастом, что тоже сомнительно, ведь это реестр заёмщиков банка. Со всеми этими проблемами предстоит разобраться.

Давайте начнём. И начнём с самого "проблемного" столбца.

## Аномалии

Итак, в столбце со стажем мы видели и отрицательные значения, и подозрительно большие, и пропуски. Со всем стоит разобраться.

In [ ]:
df['days_employed']

Давайте посмотрим на положительные и отрицательные значения отдельно. Оценим основные метрики.

In [ ]:
df[df['days_employed']>0]['days_employed'].describe()

In [ ]:
df[df['days_employed']<0]['days_employed'].describe()

Положительные значения очень большие.

Отрицательных значений очень много, и диапазон их изменения достаточно большой. Посмотрим на него в годах, так для нас будет понятнее.

In [ ]:
df[df['days_employed']<0]['days_employed'].describe() / 365

От -0.066 до -50 лет. Если бы эти значения были положительными, это очень даже похоже на реальный стаж.

Следует оценить распределение данных, как они лежат на нашей числовой прямой. Для этого нарисуем гитограмму, воспользовавшись методом `hist`.

In [ ]:
df[df['days_employed']<0]['days_employed'].hist()

На гистограмме все наши данные раскладываются в "корзины". По умолчанию их десять. Весь диапазон делится на равные интервалы, а затем подсчитывается, сколько наблюдений попало в тот или иной интервал, что и отображается на оси y. То есть для наших данных от 0 до примерно -2000 больше восьми тысяч наблюдений, от -2000 до -3500 около четырёх тысяч и так далее.

Что можно оценить по гитограмме? Тут мы видим, что количество наблюдений постепенно снижается по мере удаления от нуля. Такое распределение является распределением Пуассона. Вообще, очень хорошо, если мы видим, что данные соответствуют нормальному распределению - тогда открывается большой простор для применения различных статистических методов. Но и распределение Пуассона тоже хорошо, ведь это по сути обрезанное нормальное распределение. А для стажа распределение Пуассона является естественным, ведь в реальности действительно больше людей с маленьким стажем, а с увеличеснием стажа количество таких работников уменьшается.

Вот только наше распределение отрицательно. А если мы его повернём относительно нуля, то всё будет прекрасно.

Очень важно всегда пытаться понять, почему возникла та или иная ошибка. Из-за чего стаж мог стать отрицательным? Ваши версии?

Моя версия состоит в том, что вычисления вёл компьютер и просто вычитал из начальной даты стажа текущую или финальную. И даже могу попытаться убедить вас в этом:

In [ ]:
pd.to_datetime('2024-01-13') - pd.to_datetime('2024-10-30')

Видите, если я вычитаю из меньшей даты большую, то в результате получается отрицательное значение. Убедил? Надеюсь, что да. В таком случае все отрицательные значения стажа можно сделать положительными.

In [ ]:
df['days_employed'] = abs(df['days_employed'])

In [ ]:
df[df['days_employed']<0]

Убедились, что отрицательных не осталось.

Давайте разбираться с положительными. Помним, что они все больше 300 тысяч. Посмотрим на них в годах для лучшего понимания.

In [ ]:
df[df['days_employed']>0]['days_employed'].describe() / 365

Да уж! Люди, которые работают от 900 до 1100 лет. Настоящие долгожители! С данными точно не всё в порядке. Оценим распределение.

In [ ]:
df[df['days_employed']>300000]['days_employed'].hist(bins=30)

Ни нормального, ни Пуассона. Тут практически равномерное распределение, какая-то однородная группа. Очень странно. Посмотрим, сколько у нас таких данных.

In [ ]:
df[df['days_employed']>300000].shape

In [ ]:
f"Аномальных данных {df[df['days_employed']>300000].shape[0] / df.shape[0] * 100:.0f}%"

Напоминаю, что метод `shape` возвращает кортеж из вдух значений - количество строк и количество столбцов датафрейма.

А в f-строке мы считаем процент аномальных данных от объема всего датасета, применяя форматирование для красивого вывода.

И что же мы видим? Таких данных у нас 16%. Это достаточно много. Что с ними делать? Удалить? Не советую. Лучше всего не удалять вообще данные, ведь всё это информация, которая может быть полезна при анализе или создании моделей. Конечно, бывают очень плохие данные и их приходится убрать. В таком случае старайтесь не удалять больше 5% данных суммарно. Конечно, когда данные совсем плохие, приходится удалять и больше, но старайтесь избегать этого. Лучше попытаться понять ошибку и как-то исправить её, чем терять большое количество полезных данных.

В чём может быть проблема здесь? Как вы думаете?

Предположу, что данные записаны не в днях, а почему-то в часах. Посмотрим, как эти данные тогда будут выглядеть в годах.

In [ ]:
df[df['days_employed']>300000]['days_employed'].describe() / 365 / 24

Стаж от 37 до 45 лет выглядит теперь очень реальным. Но почему только такой большой? Почему нет стажа меньше? Выглядит несколько подозрительно, следует разбираться дальше.

Когда мы изучаем данные одного столбца, мы не должны забывать, что эти данные существуют в датасете, а не отдельно. Соответственно, они могут быть связаны с другими данными, и эта связь может помочь нам решить проблему или понять причину её возникновения.

Какие столбцы из имеющихся в нашем датасете могут быть связаны со стажем? Да, можно найти несколько, если почти не все. Но давайте выберем что-нибудь наиболее близкое.

Я бы взял тип занятости, так как оба этих параметра напрямую связанны именно с работой. Давайте с помощью метода `unique` посмотрим на уникальные значения этого столбца.

In [ ]:
df['income_type'].unique()

У нас есть восемь разных типов занятости, давайте теперь посмотрим на стаж в разрезе этих типов. Это мы сделаем с помощью метода `groupby`.

In [ ]:
df.groupby('income_type')

Сам по себе метод не выдаст вам готовый результат. Он лишь разберет ваш датасет на несколько датасетов по указанному признаку. А вот чтобы получить результат, требуется уже напрямую указать, что вы хотите увидеть. В нашем случае мы хотим посмотреть основные метрики для стажа.

In [ ]:
df.groupby('income_type')['days_employed'].describe()

Это уже интересно. Теперь мы видим, что большие значения стажа соответствуют только двум категориям. Это упрощает объяснение возникновения ошибки. Более того, это даже объясняет разброс аномального стажа в диапазоне 37-45 лет. пенсионерам такой стаж вполне соответствует.

А вот безработных всего двое. А студентов, предпринимателей и в декрете вообще по одному. Такие данные при анализе могут сослужить плохую службу - они никак не отражают свою группу. Если среди десяти тысяч сотрудников мы понимаем, что сможем понять и разброс, и средние по всей категории, то среди одного человека это будет затруднительно. Но у нас есть и хорошая новость - у нас всего пять таких человек, очень мало по сравнению с размером всего датасета. Поэтому в данном случае мы можем легко их удалить, и наш анализ не пострадает.

Итак, по данной таблице делаем два вывода:

1. Аномальные данные только у безработных и пенсионеров.
2. 4 из 8 категорий удаляем.

Начнём обработку. Для начала отфильтруем по типу занятости. Далее мы используем метод `isin` для определения вхождения значения в данный список. И сразу убедимся, что типов останется четыре.

In [ ]:
df = df[df['income_type'].isin(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий'])]

In [ ]:
df['income_type'].unique()

Прекрасно! Теперь у нас не будет лишних малочисленных категорий.

Продолжим. Теперь нужно перевести стаж в часах в стаж в днях для пенсионеров. Для этого воспользуемся методом `apply`. Он обычно пробегает по столбцам, но мы сделаем так, чтобы он пробегал по строкам нашего датафрейма, для этого укажем смену оси `axis=1`.

Метод `apply` выполняет ту функцию, которую вы укажете, передавая в неё каждую строку датафрейма. Но нам нужно лишь небольшое преобразование, поэтому мы используем lambda-функцию, чтобы не писать полноценную отдельную.

А внутрь поместим тернарный оператор, который будет возвращать стаж, делённый на 24, если тип занятости - пенсионер, и стаж в исходном виде во всех остальных случаях.

In [ ]:
df['days_employed'] = df.apply(lambda row:
                              row['days_employed'] / 24
                              if row['income_type']=='пенсионер'
                              else row['days_employed'], axis=1)

Теперь можно посмотреть на распределение наших данных.

In [ ]:
df['days_employed'].hist()

Получилось распределение Пуассона со всплеском на пенсионерах. К тому же мы спасли очень много аномальных данных от возможного удаления. Но, в столбце ещё остались проблемы.

## Пропуски

Мы сразу обратили внимание, что в стаже есть непустые значения. Помните эту таблицу?

In [ ]:
df.info()

Пропуски можно найти и другим способом, методом `isna`, он возвращает True для пустых значений и False в иных случаях.

In [ ]:
df.isna()

Да уж, кажется, это не облегчило задачу, с тем же успехом можно искать и в самом датафрейме. Но это же теперь булево значение - правда принимается за единицу, ложь за ноль. Значит при суммировании мы как раз и получим количество пустых значений в датафрейме.

In [ ]:
df.isna().sum()

Отлично! У нас по 2173 пропуска в двух столбцах. Вызывает интерес одинаковое их количество. Давайте проверим, не в одних ли и тех же строках находятся эти пропуски.

In [ ]:
df[df['days_employed'].isna()].isna().sum()

Выше мы взяли ту часть датафрейма, где есть пропуски в стаже и посчитали пропуски уже в нём. И они все совпали! Значит, стаж и доход не указаны одновременно. Из-за чего это могло возникнуть, как думаете?

Предположим, что эти люди оформляли в банке некие "легкие" продукты и от них не требовалась информация по стажу и зарплате. Имеет место быть.

Можно взглянуть на таких людей.

In [ ]:
df[df['days_employed'].isna()]

Видно, что это люди разного возраста, с разным образованием и типом занятости, с разным семейным положением. Объединяющих факторов не видно.

Пропусков таких более двух тысяч, около 10%. Опять же, удалять такое большое количество данных не хочется - потеряем много полезной информации, ведь остальные столбцы заполнены. Следовательно, необходимо придумать, как такие пропуски заполнить.

Существуют разные способы - оставить как есть, заполнить каким-то константным значением или заглушкой или подойти более индивидуально. Рассмотреть по группам и категориям. Как поступим тут?

Надеюсь вы пришли примерно к той же идее, что и я. А почему бы не заполнить это значениями, как у людей того же возраста и типа занятости? Ведь скорее всего стаж  и зарплаты таких людей примерно одинаковы. Ну по крайней мере какие-то средние значения точно подойдут. Отличная идея! Только предлагаю по возрасту взять небольшое окно, чтобы слегка расширить выборки.

In [ ]:
for i in df[df['days_employed'].isna()].index:
    inc_type = df.loc[i, 'income_type']
    age = df.loc[i, 'dob_years']
    df_group = df[(df['income_type']==inc_type)&(df['dob_years']>=age-1)&(df['dob_years']<=age+1)]
    df.loc[i, 'days_employed'] = df_group['days_employed'].median()
    df.loc[i, 'total_income'] = df_group['total_income'].median()

В данном случае мы воспользовались обычным циклом. Но пробегались по индексам только тех строк, где есть пропуски. Во второй и третьей строке мы записываем тип занятости и возраст того человека, чьи пропущенные данные хотим восстановить. Затем выделяем группу людей с тем же типом занятости и с возрастом, отличающемся максимум на единицу. И наконец на пустые места вставляем медианные значения тех же параметров нашей группы.

Почему медианные? Да, для стажа можно взять и среднее значение, оно будет скорее всего примерно таким же. Но для зарплаты куда более предпочтительно использовать именно медиану, так как она не подвержена выбросам и лучше отражает такие данные. Конечно, если четверо зарабатывают по сто тысяч, а один - шестьсот, то в среднем у всех по двести, что явно не отражает реальность. А вот медиана в таком случае - сто тысяч, что явно реальнее.

Итак, пропуск заполнили, убедимся, что их не осталось.

In [ ]:
df.isna().sum()

Похоже, у нас был человек, которому по возрасту никто не подошёл. Но ведь он всего один, так что ничего страшного, если мы его удалим.

In [ ]:
df = df[~df['days_employed'].isna()]

In [ ]:
df.isna().sum()

Теперь у нас не осталось пропусков, здорово! Но хотелось бы обратить внимание на ещё одну особенность данных в этом столбце.

## Типы данных

Стаж в днях в нашей таблице представлен числом с плавающей точкой. Почему это могло произойти?

Осмелюсь выдвинуть версию, что компьютер мог считать стаж с точностью до часов, минут и секунд, которые и попали в дробную часть. Но я сомневаюсь, что всё, что после точки, будет иметь какое-то принципиально важное значение, стаж итак указан в днях - достаточно точно. Поэтому этот столбец можно представить целыми числами. И сделать это очень легко с помощью метода `astype`.

In [ ]:
df['days_employed'] = df['days_employed'].astype('int') # писать можно и int, и 'int', и 'int32' или 'int64', если необходимо

In [ ]:
df.info()

Теперь этот столбец представлен целыми числами. Великолепно!

Обратимся к следующим проблемам.

## Дубликаты

### Явные дубликаты

По тем или иным причинам иногда случается, что в датасете присутствуют абсолютно одинаковые строки. Такие дубликаты могут быть естественными и объяснимыми и совершенно не требующими исправления, но зачастую это всё же является ошибкой. Так, в данных банка не может быть двух одинаковых людей. И хотя в нашем датафрейме нет персональных данных или чего-то, что позволяет однозначно идентифицировать человека, мы предположим, что все люди должны быть разными и ни у кого полностью список всех параметров не совпадает.

Значит нам нужно найти дубликаты, если они есть. Делается это методом `duplicated`, который возвращает True для всех строк, которые повторяют уже встречавшиеся раньше.

In [ ]:
df.duplicated().sum()

В нашем наборе данных целых 39 дубликатов! Можно даже взглянуть на них. Ниже указан аргумент `keep=False`, он нужен, чтобы оставить в данных и оригинальную строку, и её дубликат.

In [ ]:
df[df.duplicated(keep=False)].sort_values('days_employed')

Данные разнообразные, но дубликаты нам не нужны. Удалить их тоже очень и очень просто - метод `drop_duplicates`, также укажем `inplace=True`, чтобы выполнить действие "на месте" с автоматическим перезаписываением датафрейма.

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

От явных дубликатов избавились, идём дальше.

### Неявные дубликаты

Не всегда данные повторяются один в один. Иногда они значат одно и то же, но выглядят по разному. Посмотрим на данные, сможем ли увидеть такое?

In [ ]:
df

Да, например в образовании у нас есть "среднее", а есть "Среднее". Смысл одинаковый, но для компьютера - разные вещи. Это нужно исправлять, но для начала познакомимся со всеми уникальными значениями в данном столбце.

In [ ]:
df['education'].unique()

Для каждого уровня образования у нас есть три различные записи. Многовато! Однако, здесь исправить это легко - можно привести всё к нижнему регистру!

Чтобы сделать такое в pandas, необходимо дополнительно указать, что к столбцу мы будем применять строковый метод. Тогда он применится к каждому значению этого столбца. Вот так:

In [ ]:
df['education'] = df['education'].str.lower()

In [ ]:
df['education'].unique()

отлично, с образованием справились! Где ещё могут быть неявные дубликаты?

На самом деле в любом текстовом столбце, поэтому посмотрим на все по очереди. Обратим внимание на семейное положение.

In [ ]:
df['family_status'].unique()

Тут всё хорошо, проблем нет. Идём дальше, пол.

In [ ]:
df['gender'].unique()

Тут есть какое-то странное значение, давайте посмотрим, где оно встречается.

In [ ]:
df[df['gender']=='XNA']

Всего одна строка, по которой изначальный пол человека мы никак не определим. Что сделаем? Да, удалим, одну строку можно

In [ ]:
df = df[df['gender']!='XNA']

Ну и наконец самое интересное - цель кредита.

In [ ]:
df['purpose'].unique()

Что ж, тут есть где разгуляться. Каждый писал, как мог. В итоге у нас получилось много разных фраз с одинаковым смыслом. На какие категории вы бы это разделили?

Определённо, тут есть "свадьба", "образование", "автомобиль" и "недвижимость". В последнюю можно свести всё про недвижимость и жильё. Итак, хотим только четыре категории, как же мы произведём это превращение? Отбирать вручную?

Хорошая идея, можно попытаться определять подходящую категорию по основе слова - "образ" или "авто" встречаются только в своих категориях, а значит помогут чётко их разделить.

В это раз вновь воспользуемся методом `apply`, но напишем для него полноценную функцию. Однако тут мы метод будем применять только к одному столбцу, ведь нам не нужны значения других, а значит в функцию будет передаваться не строка или столбец датафрейма, а каждое значение выбранного нами столбца.

In [ ]:
def purpose_type(value):
  if 'свад' in value:
    return 'свадьба'
  elif 'образ' in value:
    return 'образование'
  elif 'авто' in value:
    return 'автомобиль'
  else:
    return 'недвижимость'

In [ ]:
df['purpose'] = df['purpose'].apply(purpose_type)

In [ ]:
df['purpose'].unique()

Получилось! У нас осталось только четыре уникальных значения. Это явно упростит в дальнейшем работу с данными.

А теперь обратимся к следующей проблеме.

## Выбросы

Текстовые столбцы мы обработали, вернемся к числовым. Вновь взглянем на основные метрики, какие проблемы видим?

In [ ]:
df.describe()

Мы точно видим проблемы с количеством детей у людей. Это нельзя оставлять без внимания. И нулевой возраст вызывает вопросы. И на зарплату точно стоит обратить внимание, там явно что-нибудь найдётся.

Начнём с детей, посмотрим на тех, у кого их -1.

In [ ]:
df[df['children']==-1].head()

Никаких явных зависимостей.

Поскольку от -1 до 20 не так много, то можем посчитать, сколько человек у нас в датасете имеют то или иное количество детей.

In [ ]:
df['children'].value_counts()

Интересный результат, у большинства от 0 до 5 детей, но есть 47 человек с -1 ребенком и 76 с 20 детьми. Если первый выброс явно ошибочный, то со вторым чуть сложнее. Да, у человека, пожалуй, могут быть 20 детей... Но! Их тут целый 76! И почему-то от 5 до 20 никаких промежуточных значений. Поэтому такое количество детей очень и очень сомнительно.

В чем могла быть ошибка, из-за чего это возникло? Может минус повился случайно, может у двойки нечаянно приписали ноль... Но тут очень сложно выдвинуть хорошую, твёрдую версию.

А может удалить? Сколько у нас этих данных?

In [ ]:
(47+76) / df.shape[0] * 100

Всего лишь чуть больше половины процента! Да, это не один и не пять человек, но в условиях, когда мы никак не можем исправить значения, полпроцента можно и удалить.

In [ ]:
df = df[(df['children']>=0)&(df['children']<10)]

In [ ]:
df['children'].value_counts()

Теперь с детьми всё хорошо, все значения выглядят реалистично и не вызывают вопросов.

Перейдём к данным с возрастом. Обратимся к гистограмме.

In [ ]:
df['dob_years'].hist()

Зачастую гистограмма на 10 корзин не сильно помогает понять данные, хотя в данном случае справляется хорошо. Но уточним гистограмму, увеличив количество корзин с помощью агрумента `bins`.

In [ ]:
df['dob_years'].hist(bins=20)

Если к совершеннолетним людям нет никаких вопросов, то на что в банке пытялись взять кредит очень молодые клиенты? Посмотрим на них.

In [ ]:
df[df['dob_years']<10]

Да, похоже, тут точно ошибка. В таком юном возрасте и столько стажа! Семейное положение, образование...

Как будем исправлять?

Восстановить возраст по стажу? Можно попробовать. Давайте посмотрим, как зависит возраст от стажа. Для этого отсортируем стаж по возрастанию и посмотрим на скорость роста такого графика.

In [ ]:
df.sort_values('days_employed').reset_index()[['days_employed']].plot()

Тут мы видим, что примерно до 5 тысяч дней количество людей с таким стажем растёт достаточно равномерно. А вот потом чем больше стаж, тем меньше таких людей - график начинает расти быстрее. Затем мы видим скос из пенсионеров (помните, тех, что были с очень большим стажем изначально) и в конце несколько человек с очень большим стажем.

Теперь посмотрим, как при этом меняется возраст.

In [ ]:
df.sort_values('days_employed').reset_index()[['dob_years']].plot()

Да, не такое мы ожидали увидеть... Заметим, что нижняя граница (наиболее плотная её часть) похожа по динамике на график стажа, такое же ускорение роста и скосы под конец. Но ширина разброса явно не позволит нам более-менее точно оценить возраст человека по стажу.

Но вы видели, сколько у нас таких данных? Всего 100 строк. Мы уже удаляли чуть больше этого, всего полпроцента. И тут схожая величина. В сумме будет чуть больше процента, такую потерю даннух мы можем себе позволить.

In [ ]:
df = df[df['dob_years']>10]

Пропустим столбцы с булевыми значениями и категориями, там всё хорошо. У нас остался последний столбец - зарплата.

Прежде всего я считаю, что данный столбец так же не нуждается в дробной части, к чему нам такая точность? Помните, как сменить тип данных столбца?

In [ ]:
df['total_income'] = df['total_income'].astype(int)

А теперь посмотрим на основные метрики.

In [ ]:
df['total_income'].describe()

Отсюда мало чего можно почерпнуть, но если поначалу рост зарплат идёт достаточно постепенно - 20 тысяч, 100, 140, 195 на каждые 25% данных, то последний скачок со 195 до двух миллионов достаточно быстрый, возможно, там что-то интересное. Посмотрим на гистограмму.

In [ ]:
df['total_income'].hist(bins=30)

Гистограмма не будет рисовать пустое место, она всегда начинается и заканчивается там, где есть данные. Значит у нас есть данные, которые значительно отстоят от основной массы данных - те самые два миллиона.

Построим ещё один график - boxplot или "ящик с усами" (также диаграмма размаха или коробчатая диаграмма).

In [ ]:
df.boxplot('total_income', vert=False, figsize=(15,5))

О чём может рассказать этот график?

Ящик - это половина всех данных, его границы - первая и ретья квартиль (25 и 75 процентиль, то ясть отделяют 25% и 75% данных), черта посередине ящика - медиана. Расстояние между 1 и 3 квартилями называется межквартильным размахом. Длина "усов" - полтора межквартильных размаха. Усы разной длины, так как левый ограничивается минимальным значением и поэтому заканчивается раньше. Точки за границами усов - потенциальные выбросы.

Тут мы видим, что у нас очень много выбросов справа. Причём есть такие, которые отстоят очень далеко, а есть и те, что лежат ближе и кучнее. Кучность говорит о том, что, скорее всего, данные эти в пределах нормы. А вот то, что лежит сильно отдельно, вызывает вопросы. Такие данные лучше убрать.

Но по какому уровню обрезать, чтобы не удалить слишком много данных? Обратимся к методу `quantile`, он выдаёт значение, отделяющее указанную долю данных. Посмотрим на значения, отделяющие 90%, 95%, 97%, 99%.

In [ ]:
df['total_income'].quantile([0.9, 0.95, 0.97, 0.98, 0.99])

Значения растут достаточно равномерно, а значение 99% лежит в плотном месте по точкам на графике. Посмотрим ещё большие процентили.

In [ ]:
df['total_income'].quantile([0.9, 0.95, 0.97, 0.98, 0.99, 0.995, 0.997, 0.998, 0.999])

Данные начинают быстро расти, но в какой момент это происходит? Посмотрим также на график отсортированных значений.

In [ ]:
df.sort_values('total_income').reset_index()['total_income'].plot(grid=True)

Здесь можно отметить, что до полумиллиона график растёт спокойно, а затем начинает расти крайне быстро. Наверно, следует обрезать по этому значению. А лучше возьмём близкое - значение 99 перцентиля, оно отличается совсем немного. По нему и обрежем.

In [ ]:
df = df[df['total_income'] < df['total_income'].quantile(0.99)]

Вновь обратимся к графикам.

In [ ]:
df['total_income'].hist()

In [ ]:
df.boxplot('total_income', vert=False, figsize=(15,5))

Теперь и гистограмма, и "ящик с усами" выглядят очень хорошо. Это нас устраивает. Смотрите на разные графики и методы представления данных, когда решаете, что удалить, а что оставить, они могут дать вам много разносторонней информации.

## Итог

In [ ]:
df

Итак, мы обработали наши данные и получили табличку с красивыми и чистыми значениями. С такой таблицей уже можно работать и не бояться, что что-либо исказит наши выводы.

Важно, что при предобработке, как и при анализе и создании моделей, нет правильного и неправильного решения. Каждое действие очень субъективно, способы обработки очень сильно разнятся от человека к человеку. Поэтому важно понимать причины и объяснять свои способы подготовки данных.